In [66]:
import json
import pandas
import requests
import csv
import time

In [62]:
import pandas as pd

# Đường dẫn đến file .xlsx
file_path = "/home/phucnguyen/code/AHCM/HeyGen/data/small_number_questions.xlsx"

# Đọc file Excel vào DataFrame
df = pd.read_excel(file_path)

video_id_list = []

error_video = []

# Duyệt từng dòng trong DataFrame
for index, row in df.iterrows():
    try:
        print(f"Row {index}:")
        
        item_dict = row.to_dict()
        print(row.to_dict())  # Chuyển dòng thành dictionary để dễ xử lý
        
        url = "https://api.heygen.com/v2/video/generate"

        payload = {
            "caption": False,  # Caption, maybe the subtile
            "title": str(item_dict["STT"]),  # Need to change , match the index of video
            "callback_id": "string",
            "dimension": {
                "width": 1024,  # Fix
                "height": 720  # fix
            },
            "video_inputs": [      # up to 50, don't know whether it is video setting or many video at a time
                {
                    "character": {   # need to change to match  the avatar create
                        "type": "talking_photo",  # avatar (already) / talking_photo ( for custome photo)
                        "talking_photo_id": "fb3e19a8f9014884a40ee3b0ebf69c04",   # !!!!! extract
                        "scale": 1,
                        "offset": {
                            "x": 0,
                            "y": 0
                        }
                    },
                    "voice": {
                        "type": "text",
                        "voice_id": "c6fb81520dcd42e0a02be231046a8639", #str
                        "input_text": item_dict["Text"] #str
                    },   # voice id Fixx
                }
            ],
            "callback_url": "string"
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "x-api-key": "NWQ1ZGJhYjY5MGY0NGNmZDk1OWU0ZWUxNDU3YzY2YzYtMTcyNjIxNTM4Mg=="
        }

        response = requests.post(url, json=payload, headers=headers)

        print(response.text)
        
        covert_to_json =  json.loads(response.text)
        
        video_id = covert_to_json["data"]["video_id"]
        
        video_id_list +=  [{"STT": item_dict["STT"], "video_id": video_id}]
        
        print(video_id)
        
        
            
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        error_video += [{"STT": item_dict["STT"], "error": e}] 

if error_video:
    print("The following videos encountered errors and were not processed:")
    print(error_video)
        
        


Row 0:
{'STT': 1, 'Text': 'Xin chào bà con, tôi là Anh Hai Cà Mau'}
{"error": null, "data": {"video_id": "f3733b8c2956416ca857d5ef1ae40387"}}
f3733b8c2956416ca857d5ef1ae40387
Row 1:
{'STT': 2, 'Text': 'Tôi có thể giúp gì cho bà con'}
{"error": null, "data": {"video_id": "9d6feabb798f4b87a4d949437cf84bc8"}}
9d6feabb798f4b87a4d949437cf84bc8
Row 2:
{'STT': 3, 'Text': 'Cách bón phân ô em cà mau green cho cây lương thực'}
{"error": null, "data": {"video_id": "abc76acf2250465bb216d9c1cbd7250b"}}
abc76acf2250465bb216d9c1cbd7250b
Row 3:
{'STT': 4, 'Text': 'Anh Hai Cà Mau xin đưa ra hướng dẫn sử dụng cho dòng phân bón'}
{"error": null, "data": {"video_id": "a5e552adf6104e189713e950163a438b"}}
a5e552adf6104e189713e950163a438b
Row 4:
{'STT': 5, 'Text': 'Whao, chúc mừng bạn đã may mắn trúng được giải thưởng vàng'}
{"error": null, "data": {"video_id": "9b4052bbdac84582b467fb8fa80e876c"}}
9b4052bbdac84582b467fb8fa80e876c


In [70]:
print(video_id_list)

[{'STT': 1, 'video_id': 'f3733b8c2956416ca857d5ef1ae40387'}, {'STT': 2, 'video_id': '9d6feabb798f4b87a4d949437cf84bc8'}, {'STT': 3, 'video_id': 'abc76acf2250465bb216d9c1cbd7250b'}, {'STT': 4, 'video_id': 'a5e552adf6104e189713e950163a438b'}, {'STT': 5, 'video_id': '9b4052bbdac84582b467fb8fa80e876c'}]


In [72]:
# nên kiểu mỗi 10p 1 lần, quét qua hết những video chưa tải về, nếu status đã success thì tải về, xong xóa khỏi list pending



for item in video_id_list:
    stt = item["STT"]
    video_id = item["video_id"]
    try:
        url = f"https://api.heygen.com/v1/video_status.get?video_id={video_id}"

        success = 0
        time_count = 0

        while success ==0 and time_count <= 240: # if wait more than 4 minutes
            
            headers = {
                "accept": "application/json",
                "x-api-key": "NWQ1ZGJhYjY5MGY0NGNmZDk1OWU0ZWUxNDU3YzY2YzYtMTcyNjIxNTM4Mg=="
            }

            process_response = requests.get(url, headers=headers)
            print("tơi day chua", process_response)
            process_response_json = json.loads(process_response.text)
            print(process_response_json)
            if process_response_json["data"]["status"] == "completed":
                success = 1
                
                video_url = process_response_json["data"]["video_url"]
                
                print(video_url)
                
                output_folder = "downloads/"
                output_file = str(item_dict["STT"]) + ".mp4" # change to the name we want

                # Tạo thư mục nếu chưa tồn tại
                import os
                os.makedirs(output_folder, exist_ok=True)

                # Tải video
                response = requests.get(video_url, stream=True)
                if response.status_code == 200:
                    with open(os.path.join(output_folder, output_file), "wb") as file:
                        for chunk in response.iter_content(chunk_size=1024):  # Tải từng chunk
                            file.write(chunk)
                    print("Video downloaded successfully!")
                else:
                    print("Failed to download video. Status code:", response.status_code)
            else: 
                time.sleep(10)
                time_count += 10
                
        if success == 0:  # take out video that not process
            error_video += {"STT": item_dict["STT"], "error": "Maximum Time Exceed" }
    except Exception as e:
        print(f"Error processing video {stt}: {e}")
        error_video += {"STT": stt, "error": e}

tơi day chua <Response [200]>
{'code': 100, 'data': {'callback_id': 'string', 'caption_url': '', 'created_at': 1733908619, 'duration': 4.25075, 'error': None, 'gif_url': 'https://resource2.heygen.ai/video/gifs/f3733b8c2956416ca857d5ef1ae40387.gif', 'id': 'f3733b8c2956416ca857d5ef1ae40387', 'status': 'completed', 'thumbnail_url': 'https://files2.heygen.ai/aws_pacific/avatar_tmp/5d5dbab690f44cfd959e4ee1457c66c6/f3733b8c2956416ca857d5ef1ae40387.jpeg?Expires=1734515334&Signature=NzkfLCPmkXhiAvPvrxt0AhgZF19dH~4EIKnQBHnLdEbO4ASne721poYZ1RP5f8aOUvTLmkuSoNfHt4AE0kj92hD5JSoOPq-J1bJHIFXidCzoLon6hjaNw8qHCbaJMSuPLe7k9wo6mQArL9z1-xQOawtZF8g53Vy2BXb5bIqgbza4zFtAef9kRMO7pCOkuMucDSIXHeaidC3~C95-jUs6vWbOtRPEj5eF90-QO9qFMvPXGZrxWOh~VJen9KnFpABrrhwBndqUj2c930C8jsXrYRMNvccSZJ3IVWDa6~s0dRisGlhBwC-Jdu85WVG8Bz3YFzrGwMZyThK~B4-cJoFJvw__&Key-Pair-Id=K38HBHX5LX3X2H', 'video_url': 'https://files2.heygen.ai/aws_pacific/avatar_tmp/5d5dbab690f44cfd959e4ee1457c66c6/f3733b8c2956416ca857d5ef1ae40387.mp4?Expires=173451

In [69]:
import os

# Get the current directory of the script
#current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = os.getcwd()

# Navigate to folder `b` from folder `a`
path_to_b = os.path.join(current_dir, "../data")

# Example: Access a file in folder `b`
file_in_b = os.path.join(path_to_b, "small_number_questions.xlsx")

# Print the path
print(f"Path to the file in 'b': {file_in_b}")

# Check if the file exists
if os.path.exists(file_in_b):
    df = pd.read_excel(file_in_b)
    for index, row in df.iterrows():
        print(index, row)
else:
    print("File does not exist.")

Path to the file in 'b': /home/phucnguyen/code/AHCM/HeyGen/code/../data/small_number_questions.xlsx
0 STT                                          1
Text    Xin chào bà con, tôi là Anh Hai Cà Mau
Name: 0, dtype: object
1 STT                                 2
Text    Tôi có thể giúp gì cho bà con
Name: 1, dtype: object
2 STT                                                     3
Text    Cách bón phân ô em cà mau green cho cây lương ...
Name: 2, dtype: object
3 STT                                                     4
Text    Anh Hai Cà Mau xin đưa ra hướng dẫn sử dụng ch...
Name: 3, dtype: object
4 STT                                                     5
Text    Whao, chúc mừng bạn đã may mắn trúng được giải...
Name: 4, dtype: object


In [ ]:
url = "https://api.heygen.com/v2/video/generate"

payload = {
    "caption": False,  # Caption, maybe the subtile
    "title": "1",  # Need to change , match the index of video
    "callback_id": "string",
    "dimension": {
        "width": 1024,  # Fix
        "height": 720  # fix
    },
    "video_inputs": [      # up to 50, don't know whether it is video setting or many video at a time
        {
            "character": {   # need to change to match  the avatar create
                "type": "talking_photo",  # avatar (already) / talking_photo ( for custome photo)
                "talking_photo_id": "fb3e19a8f9014884a40ee3b0ebf69c04",   # !!!!! extract
                "scale": 1,
                "offset": {
                    "x": 0,
                    "y": 0
                }
            },
            "voice": {
                "type": "text",
                "voice_id": "c6fb81520dcd42e0a02be231046a8639", #str
                "input_text": "Xin chào, tôi là Anh Hai Cà Mau" #str
            },   # voice id Fixx
        }
    ],
    "callback_url": "string"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "x-api-key": "NWQ1ZGJhYjY5MGY0NGNmZDk1OWU0ZWUxNDU3YzY2YzYtMTcyNjIxNTM4Mg=="
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)



{"error": null, "data": {"video_id": "057c38c37b8a4efda0027ab3940e1bda"}}


In [7]:
covert_to_json =  json.loads(response.text)
covert_to_json

{'error': None, 'data': {'video_id': '057c38c37b8a4efda0027ab3940e1bda'}}

In [8]:
video_id = covert_to_json["data"]["video_id"]
video_id

'057c38c37b8a4efda0027ab3940e1bda'

In [31]:
url = f"https://api.heygen.com/v1/video_status.get?video_id={video_id}"

success = 0
time_count = 0

while success ==0 and time_count <= 240: # if wait more than 4 minutes
    
    headers = {
        "accept": "application/json",
        "x-api-key": "NWQ1ZGJhYjY5MGY0NGNmZDk1OWU0ZWUxNDU3YzY2YzYtMTcyNjIxNTM4Mg=="
    }

    process_response = requests.get(url, headers=headers)

    process_response_json = json.loads(process_response.text)

    if process_response_json["message"] == "Success":
        success = 1
        
        video_url = process_response_json["data"]["video_url"]
        
        print(video_url)
        
        output_folder = "downloads/"
        output_file = "1.mp4" # change to the name we want

        # Tạo thư mục nếu chưa tồn tại
        import os
        os.makedirs(output_folder, exist_ok=True)

        # Tải video
        response = requests.get(video_url, stream=True)
        if response.status_code == 200:
            with open(os.path.join(output_folder, output_file), "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):  # Tải từng chunk
                    file.write(chunk)
            print("Video downloaded successfully!")
        else:
            print("Failed to download video. Status code:", response.status_code)
    else: 
        time.sleep(5)
        time_count += 5
        
        
        

https://files2.heygen.ai/aws_pacific/avatar_tmp/5d5dbab690f44cfd959e4ee1457c66c6/057c38c37b8a4efda0027ab3940e1bda.mp4?Expires=1734507076&Signature=U8nnTO4UwF2PjQzqXXHNks6tRZLa4wqfb5Zh5f8qrkcmeOezhXY~XXTLEC~Ni9in8pdmOxSTVWQprt9F12WWBPxsCHiJnb8N5TGaNqJS1orGeG2XWHlGWDJPRxti5HUBqp2ZS3HWg7R3aCx7Kp4hEeAL1GMxwYq9F1jvlrIfjA3OVakLaBiE5h4R~G23yUL3MC9p5V9FNm2OJ9v~2HcYmzK5O3vntGgOl-VzV~W7exz9jAvzZZklGMTde2zsb~hx-GHAjqN1Ro3JAEK4~a565miPgXVAgpjbCIO05y30f8A~urZWJvntH5lXqAJhJ-a5j4Lt9av-Zj8cRoKeRDLIcA__&Key-Pair-Id=K38HBHX5LX3X2H
Video downloaded successfully!
